## Benchmark 2: minimal OLS

In [2]:
# Add parent directory to path to allow import of config.py
import sys
sys.path.append('..')
import config as conf
import functions.plotting_functions as plot_funcs

import pandas as pd
import matplotlib
import statsmodels.api as sm

print(f'Python: {sys.version}')
print()
print(f'Pandas {pd.__version__}')
print(f'Matplotlib: {matplotlib.__version__}') # type: ignore
print(f'Statsmodels: {sm.__version__}')

# Replace matplotlib with pyplot interface
del matplotlib
import matplotlib.pyplot as plt

Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:24:10) [GCC 9.4.0]

Pandas 1.4.3
Matplotlib: 3.6.3
Statsmodels: 0.13.2


In [3]:
# Read zipped csv into pandas dataframe
training_df = pd.read_csv(f'{conf.DATA_PATH}/train.csv.zip', compression='zip')

# Set dtype on first day of month column
training_df['first_day_of_month'] =  pd.to_datetime(training_df['first_day_of_month'])

# Print out some metadata and sample rows
training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   row_id                 122265 non-null  object        
 1   cfips                  122265 non-null  int64         
 2   county                 122265 non-null  object        
 3   state                  122265 non-null  object        
 4   first_day_of_month     122265 non-null  datetime64[ns]
 5   microbusiness_density  122265 non-null  float64       
 6   active                 122265 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 6.5+ MB


In [14]:
# Sort each county by timepoint and take the first 3 (most recent) rows from each
recent_values_df = training_df.sort_values('first_day_of_month', ascending=False).groupby('cfips').head(3)

# Sort by cfips, then timepoint
recent_values_df.sort_values(['cfips', 'first_day_of_month'], inplace=True)

# Clean up
recent_values_df.reset_index(inplace=True, drop=True)

# Take a look
recent_values_df.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2022-08-01,1001,Autauga County,Alabama,2022-08-01,3.423852,1455
1,1001_2022-09-01,1001,Autauga County,Alabama,2022-09-01,3.442677,1463
2,1001_2022-10-01,1001,Autauga County,Alabama,2022-10-01,3.463856,1472
3,1003_2022-08-01,1003,Baldwin County,Alabama,2022-08-01,8.491150,14545
4,1003_2022-09-01,1003,Baldwin County,Alabama,2022-09-01,8.341701,14289


9405